In [2]:
# Ввод необходимых библиотек
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import enum 
import scipy.optimize as optimize
import datetime
import statsmodels.api as sm
import pandas as pd
import time
import pickle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Вводим отображения чисел при выводе
np.set_printoptions(precision = 3)
# Вводим мнимую единицу i
i   = np.complex(0.0,1.0)

# Вводим класс отражающий тип опциона

class OptionType(enum.Enum):
    CALL = 1.0
    PUT = -1.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [4]:
# моделирование двухмерной коррелированной модели Хестона на основе моделирования методом Эйлера
# возвращает 2 путя
def Bates2path(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12):
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho, 6=xip, 7=muj, 8=sigmaj)
  # мгновенная кореляция между активами
  dt = T / float(NoOfSteps)
  Z1 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z2 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z3 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z4 = np.random.normal(0.0,1.0,[1, NoOfSteps])

  # Моделирование пуассоновских процессов и их амплитуд
  J1 = np.random.normal(par1[6],par1[7],[1,NoOfSteps])
  J2 = np.random.normal(par2[6],par2[7],[1,NoOfSteps])
  Pois1 = np.random.poisson(par1[5]*dt,[1,NoOfSteps])
  Pois2 = np.random.poisson(par2[5]*dt,[1,NoOfSteps])

  W1 = np.zeros([1, NoOfSteps + 1])
  W2 = np.zeros([1, NoOfSteps + 1])
  W3 = np.zeros([1, NoOfSteps + 1])
  W4 = np.zeros([1, NoOfSteps + 1])
  V1 = np.zeros([1, NoOfSteps + 1])
  V2 = np.zeros([1, NoOfSteps + 1])
  X1 = np.zeros([1, NoOfSteps + 1])
  X2 = np.zeros([1, NoOfSteps + 1])
  V1[:,0] = par1[3]
  V2[:,0] = par2[3]
  X1[:,0] = np.log(S_01)
  X2[:,0] = np.log(S_02)
  time = np.zeros([1, NoOfSteps+1])    
  EeJ1 = np.exp(par1[6] + 0.5*par1[7]*par1[7])
  EeJ2 = np.exp(par2[6] + 0.5*par2[7]*par2[7])

  for i in range(0,NoOfSteps):
       
        Z2[:,i] = par1[4] * Z1[:,i] + np.sqrt(1.0-par1[4]**2) * Z2[:,i]
        Z4[:,i] = par2[4] * rho12 * Z1[:,i] + par2[4] * np.sqrt(1.0-rho12**2) * Z3[:,i] + np.sqrt(1.0-par2[4] ** 2) * Z4[:,i]
        Z3[:,i] = rho12 * Z1[:,i] + np.sqrt(1.0-rho12**2) * Z3[:,i]

        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        W2[:,i+1] = W2[:,i] + np.power(dt, 0.5)*Z2[:,i]
        W3[:,i+1] = W3[:,i] + np.power(dt, 0.5)*Z3[:,i]
        W4[:,i+1] = W4[:,i] + np.power(dt, 0.5)*Z4[:,i]
        
        
        V1[:,i+1] = V1[:,i] + par1[0] * (par1[2] - V1[:,i]) * dt + par1[1] * np.sqrt(V1[:,i]) * (W2[:,i+1]-W2[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1],0.0)
        X1[:,i+1] = X1[:,i] + (mu[0] - 0.5 * V1[:,i] - par1[5]*(EeJ1-1)) * dt + np.sqrt(V1[:,i]) * (W1[:,i+1]-W1[:,i]) + J1[:,i] * Pois1[:,i]
        V2[:,i+1] = V2[:,i] + par2[0] * (par2[2] - V2[:,i]) * dt + par2[1] * np.sqrt(V2[:,i]) * (W4[:,i+1]-W4[:,i])
        V2[:,i+1] = np.maximum(V2[:,i+1],0.0)
        X2[:,i+1] = X2[:,i] + (mu[1] - 0.5 * V2[:,i] - par2[5]*(EeJ2-1)) * dt + np.sqrt(V2[:,i]) * (W3[:,i+1]-W3[:,i]) + J2[:,i] * Pois2[:,i]
        
        time[0][i+1] = time[0][i] + dt
  return np.exp(X1), np.exp(X2), time    

In [5]:
# функция для подсчёта выборочной корреляции между двумя путями
def EmpCorr(path1, path2):
  return np.corrcoef([np.log(path1[j + 1]/path1[j]) for j in range(len(path1) - 1)], [np.log(path2[j + 1]/path2[j]) for j in range(len(path2) - 1)])[0][1]

In [6]:
# Моделирование многомерного пути модели Бейтса с использованием модификации введённой в работе
# параметры на входет уже отсортированы
def multi_asset_path(par_model, corr_matrix, parS0, T, r, NoOfSteps, intens):
   # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho, 6=xip, 7=muj, 8=sigmaj) 
    dt = T / float(NoOfSteps)  
    V = np.zeros([len(parS0),  NoOfSteps+1])
    X = np.zeros([len(parS0),  NoOfSteps+1])
    xiEeJ = []
    for i in range(len(parS0)):
      xiEeJ.append(par_model[i][5] * (np.exp(par_model[i][6] + 0.5*par_model[i][7]*par_model[i][7]) - 1))
    
    xi_star = []
    xi_bar = []
    xi_star.append(par_model[0][5] * intens[0])
    xi_bar.append(par_model[0][5] - xi_star[0])
    for i in range(1, len(parS0) - 1):
      xi_star.append(par_model[i][5] * intens[i] - sum(xi_star))
      xi_bar.append(par_model[i][5] - sum(xi_star))

    xi_bar.append(par_model[len(parS0) - 1][5] - sum(xi_star))

    Pois1 = np.array([np.random.poisson(xi_star[i] *dt,[1,NoOfSteps + 1])[0] for i in range(len(parS0) - 1)])
    Pois2 = np.array([np.random.poisson(xi_bar[i]*dt,[1,NoOfSteps + 1])[0] for i in range(len(parS0))])
    Pois = np.array([Pois2[i] for i in range(len(parS0))])
    for i in range(len(parS0)):
      for j in range(i + 1):
        if j == len(parS0) - 1:
          break
        else:
          Pois[i] += Pois1[j]
    
    Cj = np.zeros([len(parS0),  len(parS0)])
    for i in range(len(parS0) - 1):
      for j in range(i + 1, len(parS0)):
        Cj[i, j] = min(intens[i], intens[j]) 
        Cj[j, i] = Cj[i, j]
      Cj[i, i] = 1

    Cj[len(parS0) - 1, len(parS0) - 1] = 1
    Cj = Janckel_reg(Cj)
  

    for i in range(len(parS0) - 1):
      for j in range(i + 1, len(parS0)):
        Cj[i, j] = Cj[i, j] * par_model[i][7]* par_model[j][7]
        Cj[j, i] = Cj[i, j]
      Cj[i, i] = par_model[i][7]* par_model[i][7]

    Cj[len(parS0) - 1, len(parS0) - 1] = par_model[len(parS0) - 1][7] * par_model[len(parS0) - 1][7]
    Lj = chol(Cj)
    
    muJ = np.array([par_model[i][6] for i in range(len(parS0))])
    
    J = np.array([np.random.normal(par_model[i][6], par_model[i][7],[1 ,NoOfSteps + 1])[0] for i in range(len(parS0))])
    xiEeJ = np.array(xiEeJ)
    Vint = np.zeros([len(parS0),  NoOfSteps+1])
    f = np.zeros([len(parS0), 1])
    s = np.zeros([len(parS0), 1])
    L = chol_matr(par_model, corr_matrix)
    
    rho_m = np.zeros([len(parS0), len(parS0)])

    
    for i in range(len(parS0)):
      rho_m[i, i] = par_model[i][4]

    Ds = np.zeros([len(parS0), len(parS0)])
    for i in range(len(parS0)):
      V[i, 0] = par_model[i][3]
      X[i, 0] = np.log(parS0[i])
    L_star = np.zeros([len(parS0), len(parS0)])
    for k1, i in enumerate(range(len(par_model), len(par_model) * 2)):
      for k2, j in enumerate(range(len(par_model), len(par_model) * 2)):
          L_star[k1, k2] = L[i, j]
    time = np.zeros([NoOfSteps+1])
    
    for i in range(0,NoOfSteps):
        time[i+1] = time[i] + dt
        Z = np.random.normal(0.0, 1.0, [len(parS0), 1])
        Zj = np.random.normal(0.0, 1.0, [len(parS0), 1])
        for j in range(len(parS0)):
            #V[j,i+1] = CIR_Sample(NoOfPaths, kappa, gamma, vbar, 0, dt, V[:,i])
            V[j,i+1] = QE_scheme(par_model[j][0], par_model[j][1], par_model[j][2], time[i], time[i+1], V[j, i], 1)[0][0]
            Vint[j, i] = dt * (V[j,i+1] + V[j,i])/2
            f[j] = (V[j,i+1] - V[j,i] - par_model[j][0] * par_model[j][2] * dt + par_model[j][0] * Vint[j, i]) / par_model[j][1]
            Ds[j, j] = np.sqrt(Vint[j, i])
        
        X[:, i+1] = X[:, i] + (r  - Vint[:, i]/2 - xiEeJ) * dt + np.dot(rho_m , f).reshape([1, len(parS0)])[0] + np.dot(Ds, np.dot(L_star, Z)).reshape([1, len(parS0)])[0] + (muJ +np.dot(Lj, Zj).reshape([1, len(parS0)])[0]) * Pois[:, i]
    return np.exp(X), time

In [7]:
def check_pos_def(vec):
  for i in vec:
    if i <= 0:
      return False
  return True

In [8]:
def chol(M):
  L = np.zeros([len(M[0]), len(M[0])])
  for j in range(len(M[0])):
    for i in range(j, len(M[0])):
      if i == 0 and j == 0:
        L[i, j] = np.sqrt(M[i, j])
      if i == j and i != 0 and j != 0:
        if M[i, j] - np.sum(np.array([L[i, k] ** 2 for k in range(j)])) <= 0:
          L[i, j] = 10e-5
        else:
          L[i, j] = np.sqrt(M[i, j] - np.sum(np.array([L[i, k] ** 2 for k in range(j)])))
          
      else:
        L[i, j] = (M[i, j] - np.sum(np.array([L[j, k] * L[i, k ] for k in range(j)])))/L[j, j]
  return L

In [9]:
def chol_matr(par_model, corr_matr):
  C = np.zeros([len(par_model) * 2, len(par_model) * 2])
  
  for i in range(len(par_model)):
    C[i, i] = 1
    C[i, i + len(par_model)] = par_model[i][4]
    C[i + len(par_model), i ] = C[i, i + len(par_model)] 
  for k1, i in enumerate(range(len(par_model), len(par_model) * 2)):
    C[i, i] = 1
    for k2, j in enumerate(range(len(par_model), len(par_model) * 2)):
      
      C[i, j] = corr_matr[k1][k2]
      C[j, i] = C[i, j]
  
  w, v = np.linalg.eig(C)
  
  # проверяем положительную определённость, если не выполняется, то используем метод регуляризации Джэкеля
  if check_pos_def(w) == True:
   
    return np.linalg.cholesky(C)
  else:
    Lambda = np.zeros([len(w), len(w)])
    sp = np.zeros([len(w)])
    for i in range(len(w)):
      if w[i] >= 0:
        Lambda[i][i] = w[i]
        sp[i] = w[i] 
    T = np.zeros([len(w), len(w)])
    for i in range(len(w)):
      T[i][i] = 1/(np.power(v[i], 2).dot(sp))
    
    B = np.dot(np.dot(np.sqrt(T), v), np.sqrt(Lambda))
    Cnew = np.dot(B, np.transpose(B))
    w, v = np.linalg.eig(Cnew)
    for i in range(len(par_model) *2):
        Cnew[i, i] = 1
    return chol(Cnew)

In [10]:
# вычисление параметров для QE схемы

def FirstApproach(m, s2):
    b2 = 2.0 * m * m / s2 - 1.0 + np.sqrt(2.0 * m * m / s2)*np.sqrt((2.0 * m * m / s2) - 1.0)
    b  = np.sqrt(b2)
    a  = m  / (1.0 + b2)
    return a,b

def SecondApproach(m, s2):
    c = ((s2 / (m*m)) - 1.0) / ((s2 / (m*m)) + 1.0)
    d = (1.0 - c) / m
    return c, d

def CIRCDF(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    cdf =lambda x: st.ncx2.cdf(x/c,delta,kappaBar)
    return cdf

def CIRMean(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(- kappa * (t-s)))
    kappaBar = 4.0 * kappa * v_s * np.exp(-kappa * (t-s)) / (gamma * gamma * (1.0 - np.exp(-kappa*(t-s))))
    return c * (delta + kappaBar)

def CIRVar(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    VarV = c*c*(2.0*delta+4.0*kappaBar)
    return VarV

In [11]:
# QE схема для моделирования  V(t)|V(s)
def QE_scheme(kappa, gamma, vbar, s, t, v_s, NoOfSamples):
    m  = CIRMean(kappa, gamma, vbar, s, t, v_s)
    s2 = CIRVar(kappa, gamma, vbar, s, t, v_s)
    if m < 0 or s2 < 0:
      print('параметры', kappa, gamma, vbar, s, t, v_s)
    aStar = 1.5 # параметр разделения
    if (s2/ (m * m) < aStar):

        # a и b - первый подход

        a, b = FirstApproach(m, s2)
        Z = np.random.normal(0.0, 1.0, [NoOfSamples, 1])
        
        if NoOfSamples > 1:
          Z = (Z - np.mean(Z)) / np.std(Z)
        
        A = a * np.power(b + Z, 2.0)

    else:

        # c & d - второй подход
        c, d = SecondApproach(m, s2)
        U = np.random.uniform(0.0, 1.0, [NoOfSamples, 1])
        A = 1.0 / d * np.log((1.0-c)/(1.0-U))
        A[U < c] = 0.0
    #print(A)
    return A  

In [12]:
def Janckel_reg(M):
  w, v = np.linalg.eig(M)
  if check_pos_def(w) == True:
    return M
  else:
    Lambda = np.zeros([len(w), len(w)])
    sp = np.zeros([len(w)])
    for i in range(len(w)):
      if w[i] >= 0:
        Lambda[i][i] = w[i]
        sp[i] = w[i] 
    T = np.zeros([len(w), len(w)])
    for i in range(len(w)):
      T[i][i] = 1/(np.power(v[i], 2).dot(sp))
    
    B = np.dot(np.dot(np.sqrt(T), v), np.sqrt(Lambda))
    Cnew = np.dot(B, np.transpose(B))
    return Cnew 

In [13]:
def autocallbondprice(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, N, dates, coupon, barier, autocall, nom, ins, intens):
  price = []
  count = 0
  for i in range(N):
    C = 0
    memory = 0
    paths = multi_asset_path(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, intens)
    for k in range(len(par_S_0)):
      paths[0][k, :] = paths[0][k, :] / par_S_0[k]
    for l, j in enumerate(dates):
        d = min(paths[0][:, findind(paths[1], j)])
        if d < barier and l == len(dates) - 1:
          C +=  np.exp(- r * j) * d * nom  / barier
          count += 1
        elif barier <= d <= autocall and l == len(dates) - 1:
          memory += 1
          C += np.exp(- r * j) * (memory * nom * coupon / 100 + (1 - ins) * nom)
        elif d > autocall and l == 0:
          C = np.exp(- r * j) * (2 * nom * coupon / 100 + nom)
          break
        elif d > autocall:
          memory += 1
          C += np.exp(- r * j) * (memory * nom * coupon / 100 + nom)
          break
        elif d < barier:
          memory += 1
        elif barier <= d <= autocall:
          memory += 1
          C += np.exp(- r * j) * memory * nom * coupon / 100
          memory = 0
    price.append(C)
  price = np.array(price)
  z = 1.96
  m = np.mean(price)
  s = np.std(price, ddof=1)

  return m  , round(count / N, 3), m - z * s / np.sqrt(N) , m + z * s / np.sqrt(N)

In [14]:
# Возвращает оценённый параметр vbar, цену начала периода!!!! и длину интервала
def par_teta_est(ticker):
  b_file = open(f"{ticker}HD.pkl", "rb")
  data = pickle.load(b_file)
  b_file.close()
  v = np.array(data[ticker]) 
  S0 = v[0]
  return np.sum(np.array([np.log(v[j + 1]/v[j]) for j in range(len(v) - 1)]) ** 2) / data['Time range'], S0 , data['Time range']

In [15]:
def empcorrmatr(tickers):
  num_tickers = {}
  num = []
  for ind, ticker in enumerate(tickers):
    num_tickers.update({f'{ind}': ticker})
    num.append(ind)
  empcor = np.zeros([len(num), len(num)])
  for i in range(len(num) - 1):
    b_file = open(f"{num_tickers[f'{i}']}HD.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    path1 = data[num_tickers[f'{i}']]
    for j in range(i + 1, len(num)):
      b_file = open(f"{num_tickers[f'{j}']}HD.pkl", "rb")
      data = pickle.load(b_file)
      path2 = data[num_tickers[f'{j}']]
      b_file.close()
      empcor[i, j] = EmpCorr(path1, path2)
      empcor[j, i] = empcor[i, j]
  for i in range(len(num)):
    empcor[i, i] = 1
  return empcor

In [28]:
def sort_tickers(tickers):
  voc = {}
  b_file = open("Intensities.pkl", "rb")  
  jum = pickle.load(b_file)
  b_file.close()
  for i in tickers:
    b_file = open(f"{i}.pkl", "rb")  
    dt = pickle.load(b_file)
    b_file.close()
    voc.update({i: dt['Parameters Bates'][5] * jum[i]})
  sorted_values = sorted(voc.values()) 
  final_int = []
  sorted_tickers = []
  for i in sorted_values:
      for k in voc.keys():
          if voc[k] == i:
              sorted_tickers.append(k)
              final_int.append(jum[k])
              break
              
  return sorted_tickers, final_int
 

In [17]:

def Bates2pathmod(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12, intens):
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho, 6=xip, 7=muj, 8=sigmaj)
  # мгновенная кореляция между активами
  dt = T / float(NoOfSteps)
  Z1 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z2 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z3 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z4 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  xi_star = []
  xi_bar = []
  xi_star.append(par1[5] * intens[0])
  xi_bar.append(par1[5] - xi_star[0])
  for i in range(1, 1):
    xi_star.append(par2[5] * intens[i] - sum(xi_star))
    xi_bar.append(par2[5] - sum(xi_star))

  xi_bar.append(par2[5] - sum(xi_star))

  Pois1 = np.array([np.random.poisson(xi_star[i] *dt,[1,NoOfSteps + 1])[0] for i in range(1)])
  Pois2 = np.array([np.random.poisson(xi_bar[i]*dt,[1,NoOfSteps + 1])[0] for i in range(2)])
  Pois = np.array([Pois2[i] for i in range(2)])
  for i in range(2):
    for j in range(i + 1):
      if j == 1:
        break
      else:
        Pois[i] += Pois1[j]
  
  Cj = np.zeros([2,  2])
  for i in range(1):
    for j in range(i + 1, 2):
      Cj[i, j] = min(intens[i], intens[j]) 
      Cj[j, i] = Cj[i, j]
    Cj[i, i] = 1

  Cj[1, 1] = 1
  Cj = Janckel_reg(Cj)
  Cj[0, 1] = Cj[0, 1] * par1[7]* par2[7]
  Cj[1, 0] = Cj[0, 1]
  Cj[0, 0] = par1[7]* par1[7]
  Cj[1, 1] = par2[7] * par2[7]

  Lj = chol(Cj)

  mu1 = par1[6]
  mu2 = par2[6]
  # Моделирование пуассоновских процессов и их амплитуд

  W1 = np.zeros([1, NoOfSteps + 1])
  W2 = np.zeros([1, NoOfSteps + 1])
  W3 = np.zeros([1, NoOfSteps + 1])
  W4 = np.zeros([1, NoOfSteps + 1])
  V1 = np.zeros([1, NoOfSteps + 1])
  V2 = np.zeros([1, NoOfSteps + 1])
  X1 = np.zeros([1, NoOfSteps + 1])
  X2 = np.zeros([1, NoOfSteps + 1])
  V1[:,0] = par1[3]
  V2[:,0] = par2[3]
  X1[:,0] = np.log(S_01)
  X2[:,0] = np.log(S_02)
  time = np.zeros([1, NoOfSteps+1])    
  EeJ1 = np.exp(par1[6] + 0.5*par1[7]*par1[7])
  EeJ2 = np.exp(par2[6] + 0.5*par2[7]*par2[7])

  for i in range(0,NoOfSteps):
       
        Z2[:,i] = par1[4] * Z1[:,i] + np.sqrt(1.0-par1[4]**2) * Z2[:,i]
        Z4[:,i] = par2[4] * rho12 * Z1[:,i] + par2[4] * np.sqrt(1.0-rho12**2) * Z3[:,i] + np.sqrt(1.0-par2[4] ** 2) * Z4[:,i]
        Z3[:,i] = rho12 * Z1[:,i] + np.sqrt(1.0-rho12**2) * Z3[:,i]

        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        W2[:,i+1] = W2[:,i] + np.power(dt, 0.5)*Z2[:,i]
        W3[:,i+1] = W3[:,i] + np.power(dt, 0.5)*Z3[:,i]
        W4[:,i+1] = W4[:,i] + np.power(dt, 0.5)*Z4[:,i]
        Zj1 = np.random.normal(0.0, 1.0, [1, 1])
        Zj2 = np.random.normal(0.0, 1.0, [1, 1])
        V1[:,i+1] = V1[:,i] + par1[0] * (par1[2] - V1[:,i]) * dt + par1[1] * np.sqrt(V1[:,i]) * (W2[:,i+1]-W2[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1],0.0)
        X1[:,i+1] = X1[:,i] + (mu[0] - 0.5 * V1[:,i] - par1[5]*(EeJ1-1)) * dt + np.sqrt(V1[:,i]) * (W1[:,i+1]-W1[:,i]) + (mu1 + Zj1 * Lj[0][0]) * Pois[0][i]
        V2[:,i+1] = V2[:,i] + par2[0] * (par2[2] - V2[:,i]) * dt + par2[1] * np.sqrt(V2[:,i]) * (W4[:,i+1]-W4[:,i])
        V2[:,i+1] = np.maximum(V2[:,i+1],0.0)
        X2[:,i+1] = X2[:,i] + (mu[1] - 0.5 * V2[:,i] - par2[5]*(EeJ2-1)) * dt + np.sqrt(V2[:,i]) * (W3[:,i+1]-W3[:,i]) + (mu2 + Zj1 * Lj[1][0] + Zj2 * Lj[1][1]) * Pois[1][i]
        
        time[0][i+1] = time[0][i] + dt
  return np.exp(X1), np.exp(X2), time    

In [18]:
# функция для подсчёта ожидаемой выборочной корреляции
def E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12, intens):
  s = 0
  for i in range(N):
    a = Bates2pathmod(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12, intens)
    s += EmpCorr(a[0][0], a[1][0])
  return s / N

In [ ]:
N1 = [10, 20, 30, 50, 75, 100]
r = 0.0028
alltickers = [['UNH', 'CVX', 'BA'], ['NVDA', 'AMD', 'QCOM']]
for j in N1:
  for i in alltickers:
    tickers = sort_tickers(i)
    print(corr_matr_X(tickers[0], r, j, tickers[1]), tickers)

In [19]:
def findind(arr, elem, par = 4):
  for i, j in enumerate(arr):
    if round(j, par) == round(elem, par):
      return i
  return False 

In [20]:
def corrBatescalibration(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp, bounds, intens):
  def err_fun(x):
    return (E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, x, intens) - rho_emp) ** 2
  #res = optimize.minimize(fun=err_fun, x0=[x0], method='Nelder-Mead', bounds=bounds, tol = 1e-1)
  x = np.linspace(rho_emp - 0.1, 1, 60)
  #res = optimize.differential_evolution(err_fun, bounds, atol = 1e-1)
  k = 0
  zf = 10
  for j, i in enumerate(x):
    y = err_fun(x[j])
    if y <= 0.001:
      k =  j 
      zf = y
      break
    if zf > y:
      k = j
      zf = y
  #return res.x[0]
  return x[k]

In [21]:
def corr_calibration(par_model, par_S_0, N, NoOfSteps, T, mu, rho_emp_matr, intens):
  # par_model - матрица, строка i которой состоит из откалиброванных параметров модели Хестона для актива i
  # par_S_0 - вектор начальных стоимостей активов
  # rho_emp_matr - матрица исторических корреляций
  
  bounds = [(-1.0, 1.0)]
  corr_Xi_Xj = np.zeros([len(par_S_0), len(par_S_0)])
  # вычисляем матрицу мгновенных корреляций между активами
  for i in range(len(par_S_0)):
    corr_Xi_Xj[i][i] = 1
    for j in range(i + 1, len(par_S_0)):
      corr_Xi_Xj[i][j] = corrBatescalibration(N, par_model[i], par_model[j], NoOfSteps, T, np.array([mu[i], mu[j]]), par_S_0[i], par_S_0[j], rho_emp_matr[i][j], [(-1, 1)], [intens[i], intens[j]])
      corr_Xi_Xj[j][i] = corr_Xi_Xj[i][j]
  return corr_Xi_Xj

In [22]:
def par_data_final(tickers):
  Par = []
  S0 = []
  for ticker in tickers:
    b_file = open(f"{ticker}.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    Par.append(data['Parameters Bates'])
    b_file = open(f"{ticker}HD.pkl", "rb")
    data = pickle.load(b_file)
    S0.append(np.array(data[ticker])[-1:])
    b_file.close()
  return Par, S0

In [23]:
def intensities(tickers, t1, t2):
  data = []
  for i in tickers:
    p1 = datetime.datetime(t1[2], t1[1], t1[0], 23, 59)
    p2 = datetime.datetime(t2[2], t2[1], t2[0], 23, 59)
    period1 = int(time.mktime(p1.timetuple()))
    period2 = int(time.mktime(p2.timetuple()))
    df = yf.download(i, p1 , p2)
    data.append(np.array(df[['High', 'Low']]))

  jum = np.zeros([len(data), len(data)])

  for i in range(len(data[0]) - 1):
    for j in range(len(data) - 1):
      for k in range(j + 1, len(data)):
        if data[j][i, 0] < data[j][i+1, 1] and data[k][i, 0] < data[k][i+1, 1]:
          jum[j][k] += 1
        elif data[j][i, 1] > data[j][i+1, 0] and data[k][i, 1] > data[k][i+1, 0]:
          jum[j][k] += 1
      
      if data[j][i, 0] < data[j][i+1, 1] or data[j][i, 1] > data[j][i+1, 0]:
        jum[j][j] += 1
    if data[len(data) - 1][i, 0] < data[len(data) - 1][i+1, 1] or data[len(data) - 1][i, 1] > data[len(data) - 1][i+1, 0]:
        jum[len(data) - 1][len(data) - 1] += 1
    
  voc = {}
  for l, m in enumerate(tickers):
    voc.update({m : jum[l]})

  b_file = open("Jumps.pkl", "wb")  
  pickle.dump(voc, b_file)
  b_file.close()
  return jum
  

In [24]:
def corr_matr_X(tickers, r, N, intens):
  v_P = []
  S_0 = []  
  for ticker in tickers:
    k = par_teta_est(ticker)
    S_0.append(k[1])
    v_P.append(k[0])
    T = k[2]
  mu_P = []
  Par = []
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)
  for ind, ticker in enumerate(tickers):
    b_file = open(f"{ticker}.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    data['Parameters Bates'][2] = v_P[ind]
    Par.append(data['Parameters Bates'])
    d = data['Parameters Bates']
    mu_P.append(r + d[0] * (v_P[ind]-d[2]) / (d[4] * d[1]))
  empcorr = empcorrmatr(tickers)
  NoOfSteps = int(T * 252)
  return corr_calibration(Par, S_0, N, NoOfSteps, T, mu_P, empcorr, intens) , empcorr

In [29]:
N1 = 75
r = 0.0028

N2 = 10000


coupon = 4.5
barier = 0.65
autocall = 1
nom = 1000
ins = 0.01



#assets = [['CVX'], ['UNH'], ['BA'], ['NVDA'], ['QCOM'], ['AMD'], ['NVDA', 'QCOM'], ['BA', 'CVX'], ['NVDA', 'AMD', 'QCOM'], ['UNH', 'CVX', 'BA']]
assets = [['CVX', 'UNH']]
Expirations = [1, 2 , 3, 5, 10]



for j in assets:
    res = []
    tick = sort_tickers(j)
    tickers = tick[0]
    C = corr_matr_X(tickers, r, N1, tick[1])
    P = par_data_final(tickers)
    par_model = P[0]
    rho_emp_matr = C[0]
    par_S_0 = P[1]
    b_file = open("results_Bates_mod.pkl", "rb")
    res_voc = pickle.load(b_file)
    b_file.close()
    for i in Expirations:
        T = i
        NoOfSteps =  T * 252
        dates = [k/4 for k in range(1, T * 4 + 1)]
      
        results = autocallbondprice(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, N2, dates, coupon, barier, autocall, nom, ins, tick[1])
        res.append([i, results[0], results[1], results[2], results[3]])
        print(j, [i, results[0], results[1], results[2], results[3]])
    res_voc.update({"".join(j): res})
    b_file = open("results_Bates_mod.pkl", "wb")
    pickle.dump(res_voc, b_file)
    b_file.close()

['CVX', 'UNH'] [1, 1049.3998153755924, 0.182, 1045.9745237013924, 1052.8251070497925]
['CVX', 'UNH'] [2, 1055.874015712694, 0.21, 1051.4200381612375, 1060.3279932641506]
['CVX', 'UNH'] [3, 1063.5763871311053, 0.208, 1058.3012931979656, 1068.851481064245]
['CVX', 'UNH'] [5, 1095.0085251460084, 0.198, 1088.423713072615, 1101.5933372194017]
['CVX', 'UNH'] [10, 1156.0831962441673, 0.176, 1147.203489847348, 1164.9629026409866]


In [38]:
b_file = open("results_Bates_modf.pkl", "rb")  
jum = pickle.load(b_file)
b_file.close()


In [39]:
jum

{'BACVX': [[1,
   1046.9241148004078,
   0.197,
   1043.4851020312908,
   1050.3631275695247],
  [2, 1055.2588668355727, 0.215, 1050.66296151728, 1059.8547721538653],
  [3, 1063.2072790394195, 0.221, 1057.7404168379633, 1068.6741412408758],
  [5, 1090.3694332495204, 0.209, 1083.8529521220976, 1096.8859143769432],
  [10, 1134.5594593340543, 0.196, 1125.579034531903, 1143.5398841362055]],
 'BAUNH': [[1,
   1028.2728377037174,
   0.194,
   1024.233290997733,
   1032.3123844097017],
  [2, 1033.1491052255367, 0.214, 1028.18358403232, 1038.1146264187535],
  [3, 1034.649286852416, 0.219, 1029.0247652104554, 1040.2738084943767],
  [5, 1060.0634935477806, 0.204, 1053.3538187090471, 1066.773168386514],
  [10, 1094.0369986021085, 0.197, 1085.232260769976, 1102.841736434241]],
 'CVXUNH': [[1,
   1049.3998153755924,
   0.182,
   1045.9745237013924,
   1052.8251070497925],
  [2, 1055.874015712694, 0.21, 1051.4200381612375, 1060.3279932641506],
  [3, 1063.5763871311053, 0.208, 1058.3012931979656, 106